In [1]:
import folium
import json
import os
import pandas as pd

# Happiness data

In [2]:
happy_data = pd.read_csv('happy_2017_cleaned.csv')
happy_data.head()

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,...,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,_merge
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,...,NOR,578,ISO 3166-2:NO,Europe,Northern Europe,NaN,150.0,154.0,NaN,both
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,...,DNK,208,ISO 3166-2:DK,Europe,Northern Europe,NaN,150.0,154.0,NaN,both
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,...,ISL,352,ISO 3166-2:IS,Europe,Northern Europe,NaN,150.0,154.0,NaN,both
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,...,CHE,756,ISO 3166-2:CH,Europe,Western Europe,NaN,150.0,155.0,NaN,both
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,...,FIN,246,ISO 3166-2:FI,Europe,Northern Europe,NaN,150.0,154.0,NaN,both


## GeoJSON

Loading GeoJSON that we prepared in [data-cleaning.ipynb](data-cleaning.ipynb).

In [3]:
countries_geojson_filename = 'countries_cleaned.geojson'
with open('countries_cleaned.geojson', 'r') as f:
    countries_geojson = json.load(f)

## Folium map

The map is composed of layers.

In [4]:
# Initialize the map:
m = folium.Map(location=[50.05, 14.28], zoom_start=5, max_zoom=5)

# We use the Choropleth (coloured map) chart type.
folium.Choropleth(
    geo_data=countries_geojson,
    name='choropleth',
    data=happy_data,
    # What are the column names that contain the data we'll visualize?
    columns=['alpha-3', 'Happiness.Score'],
    key_on='feature.id',
    fill_color='Set1',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Happiness score',
    overlay=True
).add_to(m)

# We'll add tooltips from the GeoJSON features that we've prepared.
folium.GeoJson(countries_geojson,
               name="Happiness score",
               style_function=lambda x: {"weight":2, 'color':'black','fillColor':'Set1', 'fillOpacity':0.2},
              highlight_function=lambda x: {'weight':3, 'color':'black'},
               smooth_factor=2.0,
              tooltip=folium.GeoJsonTooltip(fields=['ADMIN', 'happiness_rank', 'happiness_score', 'gdp_per_capita'],
                                              aliases=['Country', 'Happiness rank', 'Happiness score', 'GDP per capita'], 
                                              sticky=True, style="font-family: courier new; color: steelblue;", opacity=0.8, direction='top')
              ).add_to(m)

# Add a layer of controls that allow to hide other layers.
folium.LayerControl().add_to(m)

m.save(os.path.join('output', 'folium_choropleth.html'))

See the resulting map at [output/folium_choropleth.html](output/folium_choropleth.html)